In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
import random
import json

# Multiple Choice Question Model

In [ ]:
# Example Data
data = pd.DataFrame({
    'paragraph': [
        'This is an example paragraph for an LSTM model.',
        'Another example of a paragraph for the model to learn.',
        'LSTM models are used for sequence prediction.',
        'Neural networks can learn complex patterns.',
        'Text classification can be done using LSTM.'
    ],
    'question': [
        'What is this paragraph for?',
        'What is this an example of?',
        'What are LSTM models used for?',
        'What can neural networks learn?',
        'What can be done using LSTM?'
    ],
    'answer': [
        'LSTM model',
        'Model to learn',
        'Sequence prediction',
        'Complex patterns',
        'Text classification'
    ],
    'distractor1': [
        'Decision Tree',
        'Random Forest',
        'Support Vector Machine',
        'Linear Regression',
        'Naive Bayes'
    ],
    'distractor2': [
        'Random Forest',
        'Support Vector Machine',
        'Naive Bayes',
        'Logistic Regression',
        'Decision Tree'
    ],
    'distractor3': [
        'Support Vector Machine',
        'Naive Bayes',
        'Decision Tree',
        'Random Forest',
        'Logistic Regression'
    ]
})

# Tokenization
tokenizer = Tokenizer()
# Train the tokenizer on texts
tokenizer.fit_on_texts(data['paragraph'])
tokenizer.fit_on_texts(data['question'])
tokenizer.fit_on_texts(data['answer'])
tokenizer.fit_on_texts(data['distractor1'])
tokenizer.fit_on_texts(data['distractor2'])
tokenizer.fit_on_texts(data['distractor3'])
# Get the vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
paragraph_sequences = tokenizer.texts_to_sequences(data['paragraph'])
max_sequence_length = max(len(seq) for seq in paragraph_sequences)
# Pad the paragraph sequences so that they all have the same length
padded_paragraphs = pad_sequences(paragraph_sequences, padding='post', maxlen=max_sequence_length)

# Convert targets to sequences
target_sequences = {}
for target in ['question', 'answer', 'distractor1', 'distractor2', 'distractor3']:
    target_data = tokenizer.texts_to_sequences(data[target])
    target_sequences[target] = pad_sequences(target_data, padding='post', maxlen=max_sequence_length)

# Verify that the target arrays contain the same number of samples as the padded paragraphs
for target in target_sequences:
    assert padded_paragraphs.shape[0] == target_sequences[target].shape[0], f"Number of samples in padded_paragraphs and target_{target} do not match."

# Verify that the target sequences are correctly flattened
for target in target_sequences:
    target_sequences[target] = np.expand_dims(np.argmax(target_sequences[target], axis=1), -1)

target_dict = {target + '_output': target_sequences[target] for target in target_sequences}

In [ ]:
# Define model architecture
input_shape = (max_sequence_length,)
input_layer = Input(shape=input_shape)
embedding_layer = Embedding(vocab_size, 128)(input_layer)
lstm_layer = LSTM(256, return_sequences=False)(embedding_layer)

# Output layers for each task
outputs = {}
# Iterate over each target to create a corresponding output layer
for target in ['question', 'answer', 'distractor1', 'distractor2', 'distractor3']:
    outputs[target + '_output'] = Dense(vocab_size, activation='softmax', name=target + '_output')(lstm_layer)

# Create the model
model = Model(inputs=input_layer, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print shapes for debugging
print("Shape of padded_paragraphs:", padded_paragraphs.shape)
for target in target_sequences:
    print(f"Shape of {target}:", target_sequences[target].shape)

Shape of padded_paragraphs: (5, 10)
Shape of question: (5, 1)
Shape of answer: (5, 1)
Shape of distractor1: (5, 1)
Shape of distractor2: (5, 1)
Shape of distractor3: (5, 1)


In [ ]:
# Train the model using the padded_paragraphs as input and target_dict as outputs
NUM_EPOCHS = 10
model.fit(padded_paragraphs, target_dict, epochs=NUM_EPOCHS, batch_size=10)


def sample_from_predictions(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    probabilities = np.random.multinomial(1, predictions, 1)
    return np.argmax(probabilities)


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 18.8001 - answer_output_loss: 3.7626 - distractor1_output_loss: 3.7565 - distractor2_output_loss: 3.7593 - distractor3_output_loss: 3.7590 - question_output_loss: 3.7627 - answer_output_accuracy: 0.0000e+00 - distractor1_output_accuracy: 0.0000e+00 - distractor2_output_accuracy: 0.0000e+00 - distractor3_output_accuracy: 0.0000e+00 - question_output_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 20ms/step - loss: 18.6835 - answer_output_loss: 3.7286 - distractor1_output_loss: 3.7375 - distractor2_output_loss: 3.7360 - distractor3_output_loss: 3.7386 - question_output_loss: 3.7429 - answer_output_accuracy: 1.0000 - distractor1_output_accuracy: 0.4000 - distractor2_output_accuracy: 0.6000 - distractor3_output_accuracy: 0.8000 - question_output_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 19ms/step - loss: 18.5555 - answer_output_loss: 3.6907 - distractor1_output_los

In [ ]:
def predict_from_paragraph(model, paragraph, tokenizer, max_length=20, temperature=1.0):
    sequence = tokenizer.texts_to_sequences([paragraph])
    padded_sequence = pad_sequences(sequence, maxlen=padded_paragraphs.shape[1], padding='post')
    predictions = model.predict(padded_sequence)

    generated_text = []
     # Generate text based on predictions for a maximum of max_length words
    for _ in range(max_length):
        question_prediction = sample_from_predictions(predictions['question_output'][0], temperature)
        word = tokenizer.index_word.get(question_prediction, '<UNK>')
        if word == '<UNK>':
            break
        generated_text.append(word)
        padded_sequence = pad_sequences([tokenizer.texts_to_sequences([word])[0]], maxlen=padded_paragraphs.shape[1], padding='post')
        predictions = model.predict(padded_sequence)

    return ' '.join(generated_text)

# Set new paragraph
new_paragraph = ""
# Generate predicted question, answer, and distractors for the new paragraph
predicted_question = predict_from_paragraph(model, new_paragraph, tokenizer, max_length=10, temperature=0.7)
predicted_answer = predict_from_paragraph(model, new_paragraph, tokenizer, max_length=10, temperature=0.7)
predicted_distractor1 = predict_from_paragraph(model, new_paragraph, tokenizer, max_length=10, temperature=0.7)
predicted_distractor2 = predict_from_paragraph(model, new_paragraph, tokenizer, max_length=10, temperature=0.7)
predicted_distractor3 = predict_from_paragraph(model, new_paragraph, tokenizer, max_length=10, temperature=0.7)

print(f'Pertanyaan: {predicted_question}')
print(f'Jawaban: {predicted_answer}')
print(f'Distraktor 1: {predicted_distractor1}')
print(f'Distraktor 2: {predicted_distractor2}')
print(f'Distraktor 3: {predicted_distractor3}')

1/1 [==============================] - 0s 20ms/step
Pertanyaan: model using prediction what model what a linear model model
Jawaban: model what networks vector paragraph what model model be using
Distraktor 1: model of another decision model model learn model of machine
Distraktor 2: what learn model what this model a is model complex
Distraktor 3: what model model model what what model can model a


# Save Model

In [ ]:
# Save the model in format h5
model.save('my_model.h5')
# Load the saved model from file
loaded_model = load_model('my_model.h5')
print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model('my_model.h5')

# Convert the model architecture to JSON format
model_json = loaded_model.to_json()

with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

from google.colab import files

files.download('model_architecture.json')